<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/NASDAQ_index_STOCK_Prediction_with_IND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import datetime # import the datetime module

# Define the date range
start_date = '2020-01-01'
end_date = datetime.datetime.now()


# Fetch daily data for NASDAQ index (^IXIC)
data = yf.download('^IXIC', start=start_date, end=end_date, interval='1d')

# Display the first few rows of the dataset
print(data.head())

# Step 1: Calculate Technical Indicators

# Moving Average (MA) - 50-day and 200-day
data['MA50'] = data['Close'].rolling(window=50).mean()
data['MA200'] = data['Close'].rolling(window=200).mean()

# Exponential Moving Average (EMA) - 20-day and 50-day
data['EMA20'] = data['Close'].ewm(span=20, adjust=False).mean()
data['EMA50'] = data['Close'].ewm(span=50, adjust=False).mean()

# Relative Strength Index (RSI) - 14-day
delta = data['Close'].diff(1)
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))

# Moving Average Convergence Divergence (MACD)
data['MACD'] = data['Close'].ewm(span=12, adjust=False).mean() - data['Close'].ewm(span=26, adjust=False).mean()

# Bollinger Bands - 20-day MA and 2 standard deviations
data['Bollinger_Upper'] = data['MA50'] + (data['Close'].rolling(20).std() * 2)
data['Bollinger_Lower'] = data['MA50'] - (data['Close'].rolling(20).std() * 2)

# Drop rows with NaN values caused by rolling calculations
data = data.dropna()

# Step 2: Normalize the data

# Select features to train the model
features = ['Close', 'MA50', 'MA200', 'EMA20', 'EMA50', 'RSI', 'MACD', 'Bollinger_Upper', 'Bollinger_Lower']
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[features])

# Step 3: Prepare the data for LSTM

def create_dataset(data, time_step=60):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:i+time_step, :])  # Use all columns (features)
        Y.append(data[i+time_step, 0])  # Predict only 'Close' price (first column)
    return np.array(X), np.array(Y)

time_step = 60
X, Y = create_dataset(scaled_data, time_step)

# Reshape the input to be [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

# Step 4: Split the data into training and testing sets (80% train, 20% test)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

# Step 5: Build the LSTM Model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, X.shape[2])))  # Use the number of features as input
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))  # Final output for predicting the closing price

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 6: Train the model
model.fit(X_train, Y_train, batch_size=32, epochs=100)

# Step 7: Make predictions on the test data
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(np.concatenate((predictions, np.zeros((predictions.shape[0], X_test.shape[2] - 1))), axis=1))[:, 0]  # Inverse transform the predictions

# Inverse scaling of the true test values
Y_test_actual = scaler.inverse_transform(np.concatenate((Y_test.reshape(-1, 1), np.zeros((Y_test.shape[0], X_test.shape[2] - 1))), axis=1))[:, 0]

# Step 8: Plot the results: predicted vs actual NASDAQ prices
plt.figure(figsize=(14, 6))
plt.plot(Y_test_actual, label='Actual Price')
plt.plot(predictions, label='Predicted Price')
plt.title('NASDAQ (^IXIC) Price Prediction with Indicators')
plt.xlabel('Time')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

# Predict the next day's price using the last 60 data points
last_60_data = scaled_data[-time_step:].reshape(1, time_step, X.shape[2])

# Predict the next day's price
next_day_prediction = model.predict(last_60_data)

# Inverse the normalization to get the actual price
next_day_prediction = scaler.inverse_transform(np.concatenate((next_day_prediction, np.zeros((1, X.shape[2] - 1))), axis=1))[:, 0]

print(f"Predicted NASDAQ (^IXIC) Price for the next day: {next_day_prediction[0]}")



[*********************100%***********************]  1 of 1 completed

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2020-01-02  9039.459961  9093.429688  9010.889648  9092.190430  9092.190430   
2020-01-03  8976.429688  9065.759766  8976.429688  9020.769531  9020.769531   
2020-01-06  8943.500000  9072.410156  8943.500000  9071.469727  9071.469727   
2020-01-07  9076.639648  9091.929688  9042.549805  9068.580078  9068.580078   
2020-01-08  9068.030273  9168.889648  9059.379883  9129.240234  9129.240234   

                Volume  
Date                    
2020-01-02  2862700000  
2020-01-03  2586520000  
2020-01-06  2810450000  
2020-01-07  2381740000  
2020-01-08  2472620000  
Epoch 1/100



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - loss: 0.0265
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0058
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0054
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0043
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0044
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0040
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0031
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0030
Epoch 9/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0032
Epoch 10/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 0.0036
Epoch 11/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0032
Epoch 12/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0030
Epoch 13/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0026
Epoch 14/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0025
Epoch 15/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0024
E